## Implementation of Brunel 2000 using the Hybrid LFP scheme
* Hybrid LFP scheme example script, applying the methodology with a model
implementation similar to:

* Nicolas Brunel. "Dynamics of Sparsely Connected Networks of Excitatory and
Inhibitory Spiking Neurons". J Comput Neurosci, May 2000, Volume 8,
Issue 3, pp 183-208

* Synopsis of the main simulation procedure:
1. Loading of parameterset
    a. network parameters
    b. parameters for hybrid scheme
2. Set up file destinations for different simulation output
3. network simulation
    a. execute network simulation using NEST (www.nest-initiative.org)
    b. merge network output (spikes, currents, voltages)
4. Create a object-representation that uses sqlite3 of all the spiking output
5. Iterate over post-synaptic populations:
    a. Create Population object with appropriate parameters for
       each specific population
    b. Run all computations for populations
    c. Postprocess simulation output of all cells in population
6. Postprocess all cell- and population-specific output data
7. Create a tarball for all non-redundant simulation output

* recommend use mpirun for parallel execution

In [2]:
from example_plotting import *
import brunel_alpha_nest as BN
import lfpykit
from mpi4py import MPI
import h5py
from parameters import ParameterSet
from hybridLFPy import PostProcess, Population, CachedNetwork, setup_file_dest
import neuron  # NEURON compiled with MPI must be imported before NEST/mpi4py
from time import time
from matplotlib import gridspec
import matplotlib.pyplot as plt
import os
import numpy as np
if 'DISPLAY' not in os.environ:
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.style

matplotlib.style.use('classic')


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject